## World Military Segmentation

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_excel('./data/world_military_power.xlsx')
df = data.copy()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138 entries, 0 to 137
Data columns (total 60 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   Military Strength                        138 non-null    object 
 1   Military Strength Power Index            138 non-null    float64
 2   Aircraft Strength                        138 non-null    object 
 3   Aircraft Strength value                  138 non-null    int64  
 4   Fighter/Interceptor Strength             138 non-null    object 
 5   Fighter/Interceptor Strength value       138 non-null    int64  
 6   Attack Aircraft Strength                 138 non-null    object 
 7   Attack Aircraft Strength value           138 non-null    int64  
 8   Transport Aircraft Fleet Strength        138 non-null    object 
 9   Transport Aircraft Fleet Strength value  138 non-null    int64  
 10  Trainer Aircraft Fleet                   138 non-n

In [3]:
df.sample(25)

,Military Strength,Military Strength Power Index,Aircraft Strength,Aircraft Strength value,Fighter/Interceptor Strength,Fighter/Interceptor Strength value,Attack Aircraft Strength,Attack Aircraft Strength value,Transport Aircraft Fleet Strength,Transport Aircraft Fleet Strength value,...,Total Population,Total Population value,Total Square Land Area,Total Square Land Area value,Total Coastline Coverage,Total Coastline Coverage value,Total Waterway Coverage,Total Waterway Coverage value,Total Border Coverage,Total Border Coverage value
38,Ethiopia,0.8581,Ethiopia,86,Ethiopia,24,Ethiopia,0,Ethiopia,9,...,Ethiopia,"10,83,86,391",Ethiopia,"11,04,300",NaN,NaN,Ethiopia,"1,04,300",Ethiopia,5925.0
118,Syria,0.8241,Syria,456,Syria,199,Syria,20,Syria,5,...,Syria,"1,94,54,263",Syria,"1,85,180",Syria,193,Syria,900,Syria,2363.0
89,Norway,0.5277,Norway,150,Norway,59,Norway,15,Norway,4,...,Norway,"53,72,191",Norway,"3,23,802",Norway,25148,Norway,1577,Norway,2566.0
134,Vietnam,0.3559,Vietnam,293,Vietnam,77,Vietnam,0,Vietnam,38,...,Vietnam,"9,70,40,334",Vietnam,"3,31,210",Vietnam,3444,Vietnam,17702,Vietnam,4616.0
36,El Salvador,3.1291,El Salvador,51,El Salvador,0,El Salvador,15,El Salvador,2,...,El Salvador,"61,87,271",El Salvador,21041,El Salvador,307,El Salvador,0,El Salvador,590.0
71,Mali,1.8941,Mali,29,Mali,9,Mali,0,Mali,6,...,Mali,"1,84,29,893",Mali,"12,40,192",NaN,NaN,Mali,0,Mali,7908.0
104,Sierra Leone,4.2063,Sierra Leone,5,Sierra Leone,0,Sierra Leone,0,Sierra Leone,0,...,Sierra Leone,"63,12,212",Sierra Leone,71740,Sierra Leone,402,Sierra Leone,800,Sierra Leone,1093.0
114,Sudan,1.3017,Sudan,190,Sudan,46,Sudan,38,Sudan,22,...,Sudan,"4,31,20,843",Sudan,"18,61,484",Sudan,853,Sudan,1723,Sudan,6819.0
49,India,0.0953,India,2123,India,538,India,172,India,250,...,India,"1,29,68,34,042",India,"32,87,263",India,7000,India,14500,India,13888.0
57,Japan,0.1501,Japan,1561,Japan,279,Japan,12,Japan,60,...,Japan,"12,61,68,156",Japan,"3,77,915",Japan,29751,Japan,1770,NaN,NaN
